# 目次
1. コレスキー分解
2. 線型方程式
3. lapack

# 0. 準備

In [16]:
import scipy 
import numpy as np
import pandas as pd
import pods 

In [2]:
from scipy.linalg import cholesky, solve_triangular, LinAlgError

In [23]:
from scipy.linalg.lapack import dpotrf, dtrtri, dpotrs, dpotri

In [70]:
class RBF:
    def __init__(self, variance=1., lengthscale=0.1):
        self.variance=variance
        self.lengthscale=lengthscale
        # self.r = self._euclidean_distance
        
    def K(self, X, X2=None):
        return self.variance * np.exp(-0.5 * (self._euc_dist(X, X2) / self.lengthscale)**2)
        # return self._euc_dist(X, X2)
        
    def _euc_dist(self, X, X2):
        if X2 is None:
            # print("X2 is None")
            # print(X2)
            Xsq = np.sum(np.square(X),1)
            r2 = -2.*(np.dot(X, X.T)) + (Xsq[:,None] + Xsq[None,:]) 
            r2 = np.clip(r2, 0, np.inf)
            np.fill_diagonal(r2, 0.)
            return np.sqrt(r2)
        else:
            # print(X)
            # print(X2)
            X1sq = np.sum(np.square(X),1)
            X2sq = np.sum(np.square(X2),1)
            r2 = -2.*np.dot(X, X2.T) + (X1sq[:,None] + X2sq[None,:])
            r2 = np.clip(r2, 0, np.inf)
            return np.sqrt(r2)


def generate_non_pd_mat():    
    # Create PD matrix
    A = np.random.randn(20, 100)
    A = A.dot(A.T)
    # Compute Eigdecomp
    vals, vectors = np.linalg.eig(A)
    # Set smallest eigenval to be negative with 5 rounds worth of jitter
    vals[vals.argmin()] = 0
    default_jitter = 1e-6 * np.mean(vals)
    vals[vals.argmin()] = -default_jitter * (10 ** 3.5)
    A_corrupt = (vectors * vals).dot(vectors.T)
    return A_corrupt


def custom_cholesky(A, max_tries=5):
    A = np.ascontiguousarray(A) # パフォーマンス向上 計算結果にも影響
    diag_A = np.diag(A)
    jitter = diag_A.mean() * 1e-6
    num_tries = 0
    
    try:
        L = cholesky(A, lower=True)
        return L
    except LinAlgError:
        num_tries += 1
        
    while num_tries <= max_tries and np.isfinite(jitter):
        try:
            L = cholesky(A + np.eye(A.shape[0]) * jitter, lower=True)
            return L
        except LinAlgError:
            jitter *= 10
            num_tries += 1
    
    raise LinAlgError("Matrix is not positive definite, even with jitter.")

In [71]:
np.random.seed(seed=0)
data = pods.datasets.olympic_100m_men()
X, Y = data["X"], data["Y"]
X_pred = np.linspace(X[:,0].min() - 30,
                     X[:,0].max() + 30,
                     500).reshape(-1,1)

# 1. コレスキー分解と jitter

In [17]:
A = generate_non_pd_mat()
print(A.shape)
display(pd.DataFrame(A).head())

(20, 20)


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
0,100.634267,-12.829878,4.482351,-19.397329,-17.783494,5.714426,-1.899622,11.066276,2.525088,-7.557913,4.727511,-23.376915,-4.205758,-3.800605,5.435693,6.627710,6.315328,-3.600386,6.997528,10.153726
1,-12.829878,101.085165,17.118474,4.236247,-14.336734,3.009797,6.191034,-4.108049,6.747134,3.220881,-15.741027,2.293545,6.819706,-14.612919,-15.832109,-13.620706,-9.837469,8.683392,-8.818511,8.837283
2,4.482351,17.118474,86.317184,-16.556263,-7.222409,0.223255,-1.108208,6.063799,5.978391,12.217012,6.204679,-5.121831,4.027695,-10.544675,12.572230,1.976537,5.270870,-1.293013,-9.132880,15.951917
3,-19.397329,4.236247,-16.556263,83.790026,15.517335,2.783499,-8.489778,13.410707,-2.341935,-3.365199,-1.522553,-8.369009,6.293095,-4.399580,17.077136,9.318106,6.740101,-2.777771,5.944511,12.562596
4,-17.783494,-14.336734,-7.222409,15.517335,86.684664,-19.899808,-9.147066,-9.235743,9.790238,-4.848889,2.421497,8.169066,10.343752,-5.271822,14.318169,11.506988,25.420724,1.783262,2.524700,-5.530373


In [18]:
scipy.linalg.cholesky(A)

LinAlgError: 20-th leading minor of the array is not positive definite

In [19]:
def custom_cholesky(A, max_tries=5):
    A = np.ascontiguousarray(A) 
    diag_A = np.diag(A)
    jitter = diag_A.mean() * 1e-6
    num_tries = 0
    
    try:
        L = cholesky(A, lower=True)
        return L
    except LinAlgError:
        num_tries += 1
        
    while num_tries <= max_tries and np.isfinite(jitter):
        try:
            L = cholesky(A + np.eye(A.shape[0]) * jitter, lower=True)
            return L
        except LinAlgError:
            jitter *= 10
            num_tries += 1
    
    raise LinAlgError("Matrix is not positive definite, even with jitter.")

In [21]:
display(pd.DataFrame(custom_cholesky(A)))

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
0,10.079155,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,-1.272912,10.020976,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,0.444715,1.764754,9.162978,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,-1.924499,0.178279,-1.747797,8.829205,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,-1.764383,-1.654793,-0.383877,1.330341,8.937070,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
5,0.566955,0.372367,-0.074868,0.416500,-2.110996,9.399257,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
6,-0.188470,0.593867,-0.226173,-1.059401,-0.802761,-0.603348,9.274701,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
7,1.097937,-0.270480,0.660578,1.894447,-1.120371,1.710281,-0.355114,8.792784,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
8,0.250526,0.705124,0.504488,-0.125013,1.315763,0.273342,0.859833,1.113350,9.239257,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
9,-0.749856,0.226164,1.326137,-0.286639,-0.549091,0.013889,0.185065,-0.903127,-0.985819,8.762896,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


# 2. LAPACK (Linear Algebra Package)

LAPACK は FORTRAN の線形代数の数値計算用ライブラリで scipy からも呼び出して使用できる．

丸め誤差等の問題から数式上では同等でも計算結果が微妙に異なる場合があるのでそれを実際に確認したい．

## 2.1 Is $A \backslash y == L^\top \backslash (L \backslash y)$ ?

ある $n \times n$ の実対称正定値行列 $A$ と $n \times m$ の行列 $y$ に対して

$$
A x = y
$$

を満たすような $n\times m$ の行列 $x$ を求めたい．

ここで $A = LL^\top$ のようにコレスキー分解をすると $x$ は

$$
x = L^\top \backslash (L \backslash y)
$$

のようにして計算できる． 

LAPACK には 

In [57]:
A = generate_non_pd_mat()
# b = np.random.rand(A.shape[0],1)
b = np.eye(A.shape[0])

L = custom_cholesky(A)

In [67]:
A_y, info = dpotrs(A, b)
tmp, _ = dpotrs(L, b, lower=True)
LT_L_y, _ = dpotrs(L.T, tmp)
LT_L_y_2 = solve_triangular(L.T, solve_triangular(L, b, lower=True))

In [69]:
pd.DataFrame(A)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
0,110.748087,29.855019,-0.629014,-11.370445,12.306930,-3.159057,-1.330548,-10.887535,13.471876,19.893355,-3.645594,2.872713,8.410392,1.358034,7.566669,1.731261,-12.654786,-8.265033,-3.206330,14.561509
1,29.855019,112.258425,-3.521497,7.103482,4.299530,-2.517343,2.793585,-11.372910,7.700118,5.278303,-18.083397,12.125698,-14.254361,-0.113064,8.318357,2.707071,-21.100249,-10.405138,-19.584931,-0.920524
2,-0.629014,-3.521497,129.634165,8.825905,-9.243710,-11.355297,-11.231992,-31.894120,7.363423,-10.850096,9.216262,-7.407491,11.526450,-17.227987,7.986781,4.164416,8.257501,-16.373551,10.805959,2.175566
3,-11.370445,7.103482,8.825905,110.828980,-8.880739,-17.429639,13.437493,6.273706,-0.743714,-16.601006,20.573136,-29.059809,7.561557,-6.067737,-10.679472,9.019741,-9.055559,-16.495681,0.012187,9.026892
4,12.306930,4.299530,-9.243710,-8.880739,100.916344,4.262390,-6.729069,-15.524070,0.278433,16.264590,-9.688075,8.997673,-1.854940,4.440565,-8.904841,3.891184,5.099945,-19.050997,13.728928,14.493084
5,-3.159057,-2.517343,-11.355297,-17.429639,4.262390,67.059024,-5.956783,12.106416,11.272223,3.832405,-1.019562,1.992562,-13.512322,-13.961771,1.916898,-16.047482,15.016928,7.932201,7.718536,3.711583
6,-1.330548,2.793585,-11.231992,13.437493,-6.729069,-5.956783,113.317257,-15.366987,4.645900,-8.069374,1.767272,5.916027,-2.677032,-5.644216,-12.605106,-1.728536,8.325766,-9.843340,2.550242,-5.310176
7,-10.887535,-11.372910,-31.894120,6.273706,-15.524070,12.106416,-15.366987,144.506131,1.324944,-7.190132,-11.904647,-15.198201,1.864405,-1.563985,-18.169410,2.526204,1.393592,31.704110,-13.733902,-2.671974
8,13.471876,7.700118,7.363423,-0.743714,0.278433,11.272223,4.645900,1.324944,132.030364,9.872698,1.139827,-2.647075,16.911399,17.370317,-11.942767,-3.440775,-1.151666,12.933147,-5.934665,-0.594184
9,19.893355,5.278303,-10.850096,-16.601006,16.264590,3.832405,-8.069374,-7.190132,9.872698,112.161953,0.969639,-8.209927,-4.684944,-2.580391,-0.744058,4.946681,-11.344737,-14.658563,3.830107,4.085553


半正定値行列でやってみる

In [77]:
k = RBF(variance=1., lengthscale=10.)
A = k.K(X)
b = Y
L = custom_cholesky(A)

In [78]:
A_y, info = dpotrs(A, b)
tmp, _ = dpotrs(L, b, lower=True)
LT_L_y, _ = dpotrs(L.T, tmp)
LT_L_y_2 = solve_triangular(L.T, solve_triangular(L, b, lower=True))

In [83]:
pd.DataFrame(LT_L_y - LT_L_y_2)

,0
0,-1.812357e+14
1,1.584922e+15
2,-1.233799e+16
3,2.602358e+16
4,-1.986013e+16
5,6.214801e+15
6,-5.291369e+15
7,8.954843e+15
8,-9.871940e+15
9,7.735232e+15


## 2.2 

$$
A^{-1} = L^{-\top}L^{-1}
$$

$$
A^{-1} = A \backslash I
$$

$$

$$